In [1]:
import json
import pandas as pd
from decimal import Decimal
from flask_cors import CORS, cross_origin
from werkzeug.wrappers import Request, Response
from flask import Flask, jsonify, request
import os
from datetime import timedelta, date,datetime
from os import remove

In [ ]:
app = Flask(__name__)

cors = CORS(app)
data_url = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv'  
data = pd.read_csv(data_url)

app.config['CORS_HEADERS'] = 'Content-Type'

lista = data.iloc[:,4:].columns
dataRegion = pd.melt(data, id_vars =['Country/Region','Province/State'], value_vars =lista)

groupData = dataRegion.groupby(['Country/Region','variable'])['value'].sum().reset_index()
groupData['Date'] =pd.to_datetime(groupData.variable)

def estado(df, columna):
    for i in range(1,len(df)):
        if (df[columna].loc[i] - df[columna].loc[i-1])>25:
            df.loc[i-1,'estado'] = True
            df.loc[i,'estado'] = False
        else:
            df.loc[i-1,'estado'] = False
    return df

def pais(pais,df):
    country = df[df["Country/Region"]==pais]
    country['Date'] =pd.to_datetime(country.variable)
    country = country.sort_values(by=['Date'])
    countryDf = country.reset_index()[::30].reset_index()
    total = countryDf["value"].max()
    countryDf["porcentaje"]= countryDf["value"].astype("int").div(total)*100
    countryDf = estado(countryDf,"porcentaje")
    labelY = list(countryDf.loc[:,"Date"].astype("string"))
    lenRows = countryDf.shape[0]
    valuesDf = countryDf.loc[:,"value"]
    rowGrow = countryDf[countryDf["estado"]==True]
    totalMeses = pd.DataFrame()
    for index,row in rowGrow.iterrows():
        porpais = df[df["Country/Region"]==pais].reset_index()
        index = porpais[porpais["variable"]==row["variable"]].index
        porpais = porpais.sort_values(by=['Date'])
        hotMonth = porpais.iloc[index[0]:index[0]+30]
        hotMonthY = list(hotMonth.loc[:,"Date"].astype("string"))
        hotMonthRows = hotMonth.shape[0]
        hotMonthDf = hotMonth.loc[:,"value"]
        dataFrameMes = pd.concat([pd.DataFrame(hotMonthY).reset_index(drop=True), pd.DataFrame(hotMonthDf).reset_index(drop=True)], axis=1)
        totalMeses = pd.concat([totalMeses.reset_index(drop=True), dataFrameMes.reset_index(drop=True)], axis=0)
        #datos = jsonify({'datosFecha' : {'fechas': pd.DataFrame(hotMonthY).to_json(), 'valores': pd.DataFrame(hotMonthDf).to_json()}})
        #datosPorJson.append(datos)
    totalDatos = pd.concat([pd.DataFrame(labelY).reset_index(drop=True), pd.DataFrame(valuesDf).reset_index(drop=True)], axis=1)
    totalDatos.columns = ['fechaTotal','datosTotal']
    totalMeses.columns = ['fecha','datos']
    totalDatos = totalDatos.reset_index(drop=True)
    totalMeses = totalMeses.reset_index(drop=True)
    return totalDatos,totalMeses


@app.route('/postDataCountry', methods=['POST'])
@cross_origin()
def getDataCountry():
    global groupData
    country = request.get_json()
    print(country)
    totalDatos, totalMeses = pais(country,groupData)
    print(totalDatos,totalMeses)
    return jsonify({'totalDatos': totalDatos.to_json(), 'totalMeses': totalMeses.to_json()})
    #return 'ds'
    
if __name__ == "__main__":
    from werkzeug.serving import run_simple
    run_simple('localhost', 9012, app)

 * Running on http://localhost:9012/ (Press CTRL+C to quit)
127.0.0.1 - - [04/Oct/2020 12:34:40] "OPTIONS //postDataCountry HTTP/1.1" 200 -
<ipython-input-2-17022835fd54>:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  country['Date'] =pd.to_datetime(country.variable)
127.0.0.1 - - [04/Oct/2020 12:34:41] "POST //postDataCountry HTTP/1.1" 200 -


US
   fechaTotal  datosTotal
0  2020-01-22           1
1  2020-02-21          16
2  2020-03-22       34824
3  2020-04-21      816297
4  2020-05-21     1585151
5  2020-06-20     2252655
6  2020-07-20     3831932
7  2020-08-19     5528376
8  2020-09-18     6730088          fecha    datos
0   2020-07-16  3573977
1   2020-07-17  3645987
2   2020-07-18  3708735
3   2020-07-19  3769990
4   2020-07-20  3831932
5   2020-07-21  3896363
6   2020-07-22  3967668
7   2020-07-23  4036741
8   2020-07-24  4109848
9   2020-07-25  4176302
10  2020-07-26  4231104
11  2020-07-27  4287684
12  2020-07-28  4353477
13  2020-07-29  4424779
14  2020-07-30  4492558
15  2020-07-31  4559711
16  2020-08-01  4617614
17  2020-08-02  4664943
18  2020-08-03  4709773
19  2020-08-04  4767233
20  2020-08-05  4820885
21  2020-08-06  4880525
22  2020-08-07  4938857
23  2020-08-08  4994547
24  2020-08-09  5040670
25  2020-08-10  5090430
26  2020-08-11  5137333
27  2020-08-12  5194337
28  2020-08-13  5246314
29  2020-08-14  5

127.0.0.1 - - [04/Oct/2020 12:35:05] "OPTIONS //postDataCountry HTTP/1.1" 200 -
<ipython-input-2-17022835fd54>:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  country['Date'] =pd.to_datetime(country.variable)
[2020-10-04 12:35:05,660] ERROR in app: Exception on /postDataCountry [POST]
Traceback (most recent call last):
  File "C:\Users\nacho\Anaconda3\lib\site-packages\flask\app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\nacho\Anaconda3\lib\site-packages\flask\app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\Users\nacho\Anaconda3\lib\site-packages\flask_cors\extension.py", line 165, in wrapped_function
    return cors_after_request(app.make_response(f(*a

Russia


127.0.0.1 - - [04/Oct/2020 12:36:24] "OPTIONS //postDataCountry HTTP/1.1" 200 -
127.0.0.1 - - [04/Oct/2020 12:36:24] "POST //postDataCountry HTTP/1.1" 200 -


...

127.0.0.1 - - [04/Oct/2020 12:59:18] "OPTIONS //postDataCountry HTTP/1.1" 200 -
<ipython-input-2-17022835fd54>:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  country['Date'] =pd.to_datetime(country.variable)
127.0.0.1 - - [04/Oct/2020 12:59:18] "POST //postDataCountry HTTP/1.1" 200 -


Spain
   fechaTotal  datosTotal
0  2020-01-22           0
1  2020-02-21           2
2  2020-03-22       28768
3  2020-04-21      204178
4  2020-05-21      233037
5  2020-06-20      245938
6  2020-07-20      264836
7  2020-08-19      370867
8  2020-09-18      640040          fecha   datos
0   2020-03-18   13910
1   2020-03-19   17963
2   2020-03-20   20410
3   2020-03-21   25374
4   2020-03-22   28768
5   2020-03-23   35136
6   2020-03-24   39885
7   2020-03-25   49515
8   2020-03-26   57786
9   2020-03-27   65719
10  2020-03-28   73235
11  2020-03-29   80110
12  2020-03-30   87956
13  2020-03-31   95923
14  2020-04-01  104118
15  2020-04-02  112065
16  2020-04-03  119199
17  2020-04-04  126168
18  2020-04-05  131646
19  2020-04-06  136675
20  2020-04-07  141942
21  2020-04-08  148220
22  2020-04-09  153222
23  2020-04-10  158273
24  2020-04-11  163027
25  2020-04-12  166831
26  2020-04-13  170099
27  2020-04-14  172541
28  2020-04-15  177644
29  2020-04-16  184948
30  2020-08-14  34281

127.0.0.1 - - [04/Oct/2020 12:59:33] "OPTIONS //postDataCountry HTTP/1.1" 200 -


Italy
   fechaTotal  datosTotal
0  2020-01-22           0
1  2020-02-21          20
2  2020-03-22       59138
3  2020-04-21      183957
4  2020-05-21      228006
5  2020-06-20      238275
6  2020-07-20      244624
7  2020-08-19      255278
8  2020-09-18      294932          fecha   datos
0   2020-03-18   35713
1   2020-03-19   41035
2   2020-03-20   47021
3   2020-03-21   53578
4   2020-03-22   59138
5   2020-03-23   63927
6   2020-03-24   69176
7   2020-03-25   74386
8   2020-03-26   80589
9   2020-03-27   86498
10  2020-03-28   92472
11  2020-03-29   97689
12  2020-03-30  101739
13  2020-03-31  105792
14  2020-04-01  110574
15  2020-04-02  115242
16  2020-04-03  119827
17  2020-04-04  124632
18  2020-04-05  128948
19  2020-04-06  132547
20  2020-04-07  135586
21  2020-04-08  139422
22  2020-04-09  143626
23  2020-04-10  147577
24  2020-04-11  152271
25  2020-04-12  156363
26  2020-04-13  159516
27  2020-04-14  162488
28  2020-04-15  165155
29  2020-04-16  168941

<ipython-input-2-17022835fd54>:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  country['Date'] =pd.to_datetime(country.variable)
127.0.0.1 - - [04/Oct/2020 12:59:34] "POST //postDataCountry HTTP/1.1" 200 -
